In [1]:
import requests
#selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
#
from bs4 import BeautifulSoup
from datetime import date
from time import sleep
import csv
import os
import json
import urllib.request
import pandas as pd
from datetime import date
from datetime import datetime


#https://wearpact.com/

In [2]:
#initialization of lists

brand = [] #use lowercase and replace space with '-' X
product_type = [] #can be blank, can be extracted from url or product name X
product_name = [] # X
product_description = [] # X
price = [] #original price before discount, can be blank if not on sale X
sales_price = [] #price you pay X
color = [] # X
material = [] # X All clothing on the website seems to be made from organic cotton to the 
                # point the html element saying the material has cotton in the name, so all of this is Organic Cotton
product_url = [] #url of product detail page X
picture_url = [] #url of product image X
timestamp = [] # current date
gender = [] #men, women, or kids (baby and kids are grouped together for consistency) X
new_arrival = [] #(can be blank) boolean, new -> true X

new_arrival_names = {"men":[], "women":[], "kids":[]} #holds the names of new arrivals for checking later 
                                                        #stored as a dictionary for simplicity in calling
current_date = date.today()

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.set_window_size(1120, 1000)

"""
The website has 6 sections (Women, Men, Baby, Kids, Bed, Bath) and we only need the first 4 as bed and bath aren't clothing
    All 4 have Apparel and new arrivals sections, but Women and Men have an underwear section as well
    This means that every catagory must be looped through, but this makes gender very easy (likely to classify baby as kids)
    Likely easiest to have a loop for each section (new arrivals, apparel, underwear)
"""




====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [C:\Users\813ro\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


"\nThe website has 6 sections (Women, Men, Baby, Kids, Bed, Bath) and we only need the first 4 as bed and bath aren't clothing\n    All 4 have Apparel and new arrivals sections, but Women and Men have an underwear section as well\n    This means that every catagory must be looped through, but this makes gender very easy (likely to classify baby as kids)\n    Likely easiest to have a loop for each section (new arrivals, apparel, underwear)\n"

In [3]:
#getting info from product

#everything is generated through JS on the page, therefore you cannot use beautiful soup (selenium is needed)
def getProductInfo(url):
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    
    #brand (always pact)
    prod_brand = "Pact"
    
    #name
    prod_title_div = soup.find('div', {'class':'product-title'})
    if prod_title_div == None:
        return None
    prod_title = prod_title_div.get_text()
    
    
    #product_type + gender
    prod_ol = soup.find('ol', {'class':'breadcrumb'})
    
    prod_gender = prod_ol.contents[0].get_text()
    if prod_gender == "baby":
        prod_gender = "kids"

    prod_type = prod_ol.contents[-1].get_text()
    
    #prices
    prod_price = None
    prod_sale = soup.find('div',{'class':'dollar red'})
    if prod_sale:
        prod_bundle = prod_sale.get_text()
        price_split = prod_bundle.split('$')
        prod_price = '$' + price_split[2]
        prod_sale_price = '$' + price_split[1]
    else:
        prod_sale_price = soup.find('div', {'class':'dollar'}).get_text()
    
    
    #desc
    desc_div = soup.find("div", {'class':'features'})
    desc_ul = desc_div.contents[0]
    desc_sentence = ''
    for child in desc_ul.children:
        desc_sentence += child.get_text() + ". "
    
    #Material (see explanation in previous cell)
    prod_material = "Organic Cotton"
    
    #New Arrival
    is_new = prod_title in new_arrival_names[prod_gender]
    
    #color + image url are unique, as they are a different entry but share almost all of the same data (including url)
        #therefore this really makes as many entries as there are color options (using clicks from selenium)
    color_list = soup.find('div',{'class':'product-pack-color-type'}).div.ul
    count = 1
    for child in color_list.children:
        #color name
        prod_color = child['aria-label'][6:] #takes everything past "color "
        #image url (click with selenium)
        color_btn = driver.find_element(By.XPATH, f"//div[@class='product-pack-color-type']/div/ul/li[{count}]")
        
        color_btn.click()
        sleep(0.3)
        image_tag = driver.find_element(By.XPATH, "//img[@alt='product image 0']")
        prod_image_url = image_tag.get_attribute("src")
        count += 1
        #creating df entry
        timestamp.append(current_date)
        product_url.append(url)
        brand.append(prod_brand)
        product_name.append(prod_title)
        gender.append(prod_gender)
        product_type.append(prod_type)
        price.append(prod_price)
        sales_price.append(prod_sale_price)
        product_description.append(desc_sentence)
        material.append(prod_material)
        new_arrival.append(is_new)
        color.append(prod_color)
        picture_url.append(prod_image_url)

In [4]:
#new arrivals
#this can be done 
genders = {0:"women", 1:"men", 2:'kids', 3:'kids'}
new_arrival_urls = ("https://wearpact.com/women/new", "https://wearpact.com/men/new", "https://wearpact.com/baby/new", "https://wearpact.com/kids/new")
#driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #scrolls to bottom of page
for i in range(len(new_arrival_urls)):
    gender_sort = genders[i]
    driver.get(new_arrival_urls[i])
    
    total_height = int(driver.execute_script("return document.body.scrollHeight"))
    i = 1
    while i < total_height:
        i+=5
        total_height = int(driver.execute_script("return document.body.scrollHeight"))
        driver.execute_script("window.scrollTo(0, {});".format(i))
    
    sleep(0.3)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    prod_list = soup.find('div',{'class':'product-list'})
    for product in prod_list.children:
        title_div = product.find('div',{'class':'card-style'})
        if title_div:
            new_arrival_names[gender_sort].append(title_div.get_text())
 

In [5]:
#getting links to crawl
urls = ("https://wearpact.com/women", "https://wearpact.com/men", "https://wearpact.com/baby", "https://wearpact.com/kids")
product_links = []
for section in urls:
    driver.get(section)
    total_height = int(driver.execute_script("return document.body.scrollHeight"))
    i = 1
    while i < total_height:
        i+=5
        total_height = int(driver.execute_script("return document.body.scrollHeight"))
        driver.execute_script("window.scrollTo(0, {});".format(i))
    
    sleep(0.3)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    prod_list = soup.find('div',{'class':'product-list'})
    for product in prod_list.children:
        prod_link = product.find('a',{'class':'product-link'})
        if prod_link:
            product_links.append("https://wearpact.com" + prod_link['href'])
        else:
            print("FAIL")


FAIL
FAIL


In [6]:
#all
for link in product_links:
    getProductInfo(link)

In [8]:
#creating dataframe and CSV
dataframe = pd.DataFrame(list(zip(brand, product_type, product_name, product_description, price, sales_price, color, material, product_url, picture_url, timestamp, gender, new_arrival)), columns = ["Brand", "Product Type", "Product Name", "Description", "Price", "Sales Price", "Color", "Material", "Product URL", "Picture URL", "Timestamp", "Gender", "New Arrival"])
dataframe.head()
dataframe.to_csv('wearpact.csv')